# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [3]:
df=data.groupby(['CustomerID', 'ProductName'])['Quantity'].sum().reset_index()
#df.loc[df.CustomerID ==200,['ProductName', 'Quantity']].sort_values(by='Quantity' ,ascending=False)
#Products purchased by customerID 200

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [4]:
matrix=df.pivot_table(index='ProductName', columns='CustomerID', values='Quantity', aggfunc='sum').fillna(0)
matrix.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [5]:
distances = pd.DataFrame(1/(1 + squareform(pdist(matrix.T, 'euclidean'))), index=matrix.columns, columns=matrix.columns)
distances.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [6]:
CustID=200
similarities = list(distances[CustID].sort_values(ascending=False)[1:6].index)
similarities
#Similar IDs to customerID 200

[3317, 1072, 3535, 1920, 3909]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [7]:
selected_customers=df[df['CustomerID'].isin(similarities)]
selected_customers

,CustomerID,ProductName,Quantity
1082,1072,"Appetizer - Mini Egg Roll, Shrimp",1
1083,1072,Appetizer - Sausage Rolls,1
1084,1072,"Beans - Kidney, Canned",1
1085,1072,Beef - Montreal Smoked Brisket,1
1086,1072,Beef - Striploin Aa,1
...,...,...,...
3263,3909,Wine - Charddonnay Errazuriz,2
3264,3909,Wine - Fume Blanc Fetzer,1
3265,3909,"Wine - Magnotta, Merlot Sr Vqa",1
3266,3909,Wine - Toasted Head,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [8]:
rank_products=selected_customers.groupby('ProductName')['Quantity'].sum().sort_values(ascending=False).reset_index()
rank_products[0:5]

,ProductName,Quantity
0,Soup - Campbells Bean Medley,4
1,Muffin - Carrot Individual Wrap,3
2,Bay Leaf,3
3,Pork - Kidney,3
4,"Pepper - Black, Whole",3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [9]:
# Merge the ranked products data frame with the customer product matrix on the ProductName field.
merged=rank_products.merge(matrix, on='ProductName').reset_index()
merged.head()

,index,ProductName,Quantity,33,200,264,356,412,464,477,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
0,0,Soup - Campbells Bean Medley,4,0.0,0.0,1.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,25.0,50.0,25.0,0.0,0.0,0.0,0.0
1,1,Muffin - Carrot Individual Wrap,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0
2,2,Bay Leaf,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,25.0,25.0,0.0,0.0,0.0
3,3,Pork - Kidney,3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,"Pepper - Black, Whole",3,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
merged.Quantity.isnull().sum()

0

In [12]:
#merged.loc[merged['Quantity'] !=0, [200, 'ProductName']]
#step by step
recommended=merged[['ProductName', 'Quantity', 200]].loc[merged[200]==0].sort_values(by='Quantity', ascending=False)


In [13]:
recommended.Quantity.unique()  # re check if quantities are null

array([4, 3, 2, 1], dtype=int64)

In [14]:
recommended[:5]

,ProductName,Quantity,200
0,Soup - Campbells Bean Medley,4,0.0
2,Bay Leaf,3,0.0
3,Pork - Kidney,3,0.0
5,Wanton Wrap,3,0.0
1,Muffin - Carrot Individual Wrap,3,0.0


In [15]:
list_products=list(recommended.ProductName) #(or df.index)
list_products[:5]
#Top 5 recommended products for customer ID200 - as list sorted from df

['Soup - Campbells Bean Medley',
 'Bay Leaf',
 'Pork - Kidney',
 'Wanton Wrap',
 'Muffin - Carrot Individual Wrap']

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [16]:
dict_recommendation={}
ls_custID=list(df.CustomerID.unique())

In [26]:
for customer in ls_custID:
    
    similarities = list(distances[customer].sort_values(ascending=False)[1:].head().index)
    selected_customers=df[df['CustomerID'].isin(similarities)]
    rank_products=selected_customers.groupby('ProductName')['Quantity'].sum().sort_values(ascending=False).reset_index()
    merged=rank_products.merge(matrix, on='ProductName')
    recommended=merged[['ProductName', 'Quantity', customer]].loc[merged[customer]==0].sort_values(by='Quantity', ascending=False)[:5]
  
    dict_recommendation[customer]= list(recommended.ProductName)

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [28]:
df_recom=pd.DataFrame.from_dict(dict_recommendation, orient='index').reset_index()
df_recom.columns=['CustomerID', 'Rec1', 'Rec2', 'Rec3', 'Rec4', 'Rec5']
df_recom

,CustomerID,Rec1,Rec2,Rec3,Rec4,Rec5
0,33,Butter - Unsalted,Wine - Ej Gallo Sierra Valley,Wine - Blue Nun Qualitatswein,Soup - Campbells Bean Medley,Bacardi Breezer - Tropical
1,200,Soup - Campbells Bean Medley,Bay Leaf,Pork - Kidney,Wanton Wrap,Muffin - Carrot Individual Wrap
2,264,"Fish - Scallops, Cold Smoked",Wine - Two Oceans Cabernet,Bread - Italian Roll With Herbs,Fondant - Icing,"Veal - Inside, Choice"
3,356,"Veal - Inside, Choice","Tart Shells - Sweet, 4",Pomello,Beef - Inside Round,Sardines
4,412,"Sauce - Gravy, Au Jus, Mix",Olive - Spread Tapenade,"Pepper - Black, Whole",Wine - Blue Nun Qualitatswein,Sprouts - Baby Pea Tendrils
...,...,...,...,...,...,...
995,97928,"Soup - Campbells, Lentil","Cheese - Brie,danish",Wiberg Super Cure,Pants Custom Dry Clean,Shrimp - 31/40
996,98069,Skirt - 29 Foot,Beans - Kidney White,Milk - 1%,Cheese - Taleggio D.o.p.,Yoghurt Tubes
997,98159,"Lamb - Whole, Fresh",Bread - Raisin Walnut Oval,Chips Potato Salt Vinegar 43g,Tea - English Breakfast,Curry Paste - Madras
998,98185,Pernod,Crackers - Trio,Tea - Jasmin Green,Pastry - Choclate Baked,"Peas - Pigeon, Dry"


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [30]:
distances_correlation = pd.DataFrame(1/(1 + squareform(pdist(matrix.T, 'correlation'))), 
                         index=matrix.columns, columns=matrix.columns)
distances_correlation

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.493776,0.529677,0.510360,0.506951,0.507217,0.487410,0.497932,0.499215,0.502499,...,0.493385,0.484376,0.493486,0.479867,0.499870,0.479764,0.493197,0.486457,0.491384,0.492072
200,0.493776,1.000000,0.499733,0.498575,0.482172,0.481921,0.519139,0.480599,0.499131,0.505459,...,0.504719,0.496170,0.517884,0.508954,0.492581,0.484514,0.534491,0.500253,0.484272,0.488735
264,0.529677,0.499733,1.000000,0.497579,0.513573,0.490305,0.503243,0.480936,0.502028,0.490242,...,0.508803,0.490805,0.499750,0.499162,0.517743,0.525278,0.511845,0.503500,0.515736,0.490875
356,0.510360,0.498575,0.497579,1.000000,0.490143,0.489778,0.501933,0.507803,0.482686,0.486354,...,0.499105,0.486660,0.498662,0.502228,0.494239,0.496086,0.484207,0.502204,0.506401,0.500631
412,0.506951,0.482172,0.513573,0.490143,1.000000,0.497577,0.505111,0.512470,0.493947,0.507777,...,0.488109,0.529502,0.492033,0.495264,0.504752,0.500064,0.481259,0.502418,0.505997,0.515015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.479764,0.484514,0.525278,0.496086,0.500064,0.507963,0.504739,0.499300,0.500268,0.492619,...,0.494577,0.504692,0.487604,0.510055,0.534474,1.000000,0.525583,0.508923,0.492050,0.482921
98069,0.493197,0.534491,0.511845,0.484207,0.481259,0.484219,0.496892,0.490306,0.495330,0.506038,...,0.489353,0.499742,0.493332,0.484621,0.518025,0.525583,1.000000,0.511707,0.501063,0.498897
98159,0.486457,0.500253,0.503500,0.502204,0.502418,0.499195,0.500240,0.502118,0.510333,0.502173,...,0.505074,0.522726,0.500553,0.489177,0.506341,0.508923,0.511707,1.000000,0.504367,0.495580


In [31]:
dict_recommendation={}
ls_custID=list(df.CustomerID.unique())
for customer in ls_custID:
    
    similarities = list(distances_correlation[customer].sort_values(ascending=False)[1:].head().index)
    selected_customers=df[df['CustomerID'].isin(similarities)]
    rank_products=selected_customers.groupby('ProductName')['Quantity'].sum().sort_values(ascending=False).reset_index()
    merged=rank_products.merge(matrix, on='ProductName')
    recommended=merged[['ProductName', 'Quantity', customer]].loc[merged[customer]==0].sort_values(by='Quantity', ascending=False)[:5]
  
    dict_recommendation[customer]= list(recommended.ProductName)
df_recom=pd.DataFrame.from_dict(dict_recommendation, orient='index').reset_index()
df_recom.columns=['CustomerID', 'Rec1', 'Rec2', 'Rec3', 'Rec4', 'Rec5']
df_recom

,CustomerID,Rec1,Rec2,Rec3,Rec4,Rec5
0,33,Knife Plastic - White,Muffin - Zero Transfat,Banana Turning,Crush - Cream Soda,Veal - Osso Bucco
1,200,Otomegusa Dashi Konbu,Potatoes - Idaho 100 Count,Pail With Metal Handle 16l White,Crackers - Trio,Milk Powder
2,264,"Water - Mineral, Natural",Wine - Toasted Head,Snapple - Iced Tea Peach,Muffin Batt - Blueberry Passion,Garbag Bags - Black
3,356,Cheese - Taleggio D.o.p.,"Coconut - Shredded, Sweet",Cheese - Cheddarsliced,Ocean Spray - Kiwi Strawberry,Soup Knorr Chili With Beans
4,412,Butter - Unsalted,Cake - Mini Cheesecake,"Salmon - Atlantic, Skin On",Wine - Hardys Bankside Shiraz,Gloves - Goldtouch Disposable
...,...,...,...,...,...,...
995,97928,Lettuce - Treviso,"Yogurt - Blueberry, 175 Gr",Bread - Calabrese Baguette,Extract - Lemon,Coffee - Irish Cream
996,98069,"Wine - Red, Colio Cabernet",Veal - Inside,Soupfoamcont12oz 112con,Sprouts - Baby Pea Tendrils,Peas - Frozen
997,98159,"Water, Tap",Bananas,Wine - Redchard Merritt,"Fish - Scallops, Cold Smoked",Cumin - Whole
998,98185,Squid U5 - Thailand,Cheese - Taleggio D.o.p.,"Peas - Pigeon, Dry",Salmon Steak - Cohoe 8 Oz,"Sole - Dover, Whole, Fresh"


In [32]:
distances_cityblock = pd.DataFrame(1/(1 + squareform(pdist(matrix.T, 'cityblock'))), 
                         index=matrix.columns, columns=matrix.columns)
distances_cityblock

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.008000,0.009901,0.009174,0.008403,0.008929,0.007752,0.008850,0.008621,0.008696,...,0.000564,0.000596,0.000556,0.000674,0.000542,0.000569,0.000556,0.000554,0.000476,0.000534
200,0.008000,1.000000,0.008130,0.007874,0.006993,0.007576,0.007752,0.007752,0.008065,0.008130,...,0.000563,0.000593,0.000557,0.000675,0.000538,0.000567,0.000558,0.000553,0.000473,0.000531
264,0.009901,0.008130,1.000000,0.008547,0.008850,0.008475,0.007874,0.008403,0.008621,0.008264,...,0.000566,0.000596,0.000556,0.000678,0.000543,0.000576,0.000559,0.000557,0.000479,0.000533
356,0.009174,0.007874,0.008547,1.000000,0.008000,0.008197,0.007752,0.009174,0.008065,0.008130,...,0.000564,0.000595,0.000556,0.000676,0.000540,0.000571,0.000553,0.000556,0.000478,0.000534
412,0.008403,0.006993,0.008850,0.008000,1.000000,0.008065,0.007752,0.008850,0.007937,0.008264,...,0.000561,0.000599,0.000553,0.000674,0.000540,0.000570,0.000552,0.000555,0.000477,0.000536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.000569,0.000567,0.000576,0.000571,0.000570,0.000573,0.000571,0.000573,0.000572,0.000571,...,0.000336,0.000360,0.000322,0.000381,0.000357,1.000000,0.000364,0.000351,0.000298,0.000301
98069,0.000556,0.000558,0.000559,0.000553,0.000552,0.000554,0.000553,0.000556,0.000556,0.000556,...,0.000325,0.000336,0.000322,0.000342,0.000351,0.000364,1.000000,0.000351,0.000308,0.000330
98159,0.000554,0.000553,0.000557,0.000556,0.000555,0.000556,0.000554,0.000557,0.000558,0.000556,...,0.000330,0.000367,0.000333,0.000348,0.000328,0.000351,0.000351,1.000000,0.000317,0.000315


In [33]:
dict_recommendation={}
ls_custID=list(df.CustomerID.unique())
for customer in ls_custID:
    
    similarities = list(distances_cityblock[customer].sort_values(ascending=False)[1:].head().index)
    selected_customers=df[df['CustomerID'].isin(similarities)]
    rank_products=selected_customers.groupby('ProductName')['Quantity'].sum().sort_values(ascending=False).reset_index()
    merged=rank_products.merge(matrix, on='ProductName')
    recommended=merged[['ProductName', 'Quantity', customer]].loc[merged[customer]==0].sort_values(by='Quantity', ascending=False)[:5]
  
    dict_recommendation[customer]= list(recommended.ProductName)
df_recom=pd.DataFrame.from_dict(dict_recommendation, orient='index').reset_index()
df_recom.columns=['CustomerID', 'Rec1', 'Rec2', 'Rec3', 'Rec4', 'Rec5']
df_recom

,CustomerID,Rec1,Rec2,Rec3,Rec4,Rec5
0,33,Wine - Charddonnay Errazuriz,"Oranges - Navel, 72",Bay Leaf,Pecan Raisin - Tarts,Chocolate - Dark
1,200,Scampi Tail,Pork - Kidney,Muffin - Carrot Individual Wrap,Soup - Campbells Bean Medley,Bay Leaf
2,264,Pork - Kidney,Black Currants,Bread - Italian Roll With Herbs,Wine - Two Oceans Cabernet,Muffin - Carrot Individual Wrap
3,356,Butter - Unsalted,"Veal - Inside, Choice","Nut - Chestnuts, Whole","Beets - Candy Cane, Organic",Wasabi Powder
4,412,Wine - Blue Nun Qualitatswein,Butter - Unsalted,Soup - Campbells Bean Medley,"Pepper - Black, Whole",Table Cloth 81x81 White
...,...,...,...,...,...,...
995,97928,Wine - Chardonnay South,Raspberries - Fresh,Butter - Unsalted,Tea - Herbal Sweet Dreams,"Pasta - Detalini, White, Fresh"
996,98069,Wine - Chardonnay South,Pork - Hock And Feet Attached,Bar - Granola Trail Mix Fruit Nut,"Water, Tap",Beef Wellington
997,98159,Bandage - Fexible 1x3,Soup - Campbells Bean Medley,Knife Plastic - White,Sherry - Dry,"Pepper - Black, Whole"
998,98185,Muffin - Carrot Individual Wrap,Berry Brulee,Bar - Granola Trail Mix Fruit Nut,Lamb - Ground,Beef - Texas Style Burger


In [34]:
jaccard = pd.DataFrame(1/(1 + squareform(pdist(matrix.T, 'jaccard'))), index=matrix.columns, columns=matrix.columns)
jaccard

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.514644,0.529703,0.524038,0.513043,0.516129,0.510549,0.509174,0.511211,0.513889,...,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
200,0.514644,1.000000,0.518828,0.514523,0.503704,0.512000,0.520325,0.510204,0.518828,0.519313,...,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
264,0.529703,0.518828,1.000000,0.515695,0.527027,0.515419,0.510460,0.508772,0.515419,0.511013,...,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
356,0.524038,0.514523,0.515695,1.000000,0.517094,0.510917,0.508403,0.521531,0.510823,0.513393,...,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
412,0.513043,0.503704,0.527027,0.517094,1.000000,0.512605,0.516529,0.520362,0.510373,0.515284,...,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,...,0.518987,0.525581,0.515021,0.521951,0.520362,1.000000,0.524887,0.525346,0.511628,0.504000
98069,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,...,0.516129,0.512821,0.514644,0.511013,0.526549,0.524887,1.000000,0.527027,0.517510,0.523404
98159,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,...,0.512500,0.525822,0.520000,0.513889,0.515152,0.525346,0.527027,1.000000,0.524793,0.512605


In [35]:
dict_recommendation={}
ls_custID=list(df.CustomerID.unique())
for customer in ls_custID:
    
    similarities = list(jaccard[customer].sort_values(ascending=False)[1:].head().index)
    selected_customers=df[df['CustomerID'].isin(similarities)]
    rank_products=selected_customers.groupby('ProductName')['Quantity'].sum().sort_values(ascending=False).reset_index()
    merged=rank_products.merge(matrix, on='ProductName')
    recommended=merged[['ProductName', 'Quantity', customer]].loc[merged[customer]==0].sort_values(by='Quantity', ascending=False)[:5]
  
    dict_recommendation[customer]= list(recommended.ProductName)
df_recom=pd.DataFrame.from_dict(dict_recommendation, orient='index').reset_index()
df_recom.columns=['CustomerID', 'Rec1', 'Rec2', 'Rec3', 'Rec4', 'Rec5']
df_recom

,CustomerID,Rec1,Rec2,Rec3,Rec4,Rec5
0,33,Loquat,"Cheese - Brie, Triple Creme",Blueberries,Wine - Wyndham Estate Bin 777,Scampi Tail
1,200,Muffin - Carrot Individual Wrap,"Beans - Kidney, Canned",Pomello,Chips Potato Salt Vinegar 43g,Wine - Blue Nun Qualitatswein
2,264,Sauce - Demi Glace,Wine - Two Oceans Cabernet,Flavouring - Orange,"Lamb - Pieces, Diced",Beer - Blue
3,356,"Veal - Inside, Choice",Bread - Italian Roll With Herbs,"Tart Shells - Sweet, 4",Beef - Inside Round,"Pepper - Black, Whole"
4,412,Cumin - Whole,Smirnoff Green Apple Twist,"Pepper - Black, Whole","Tart Shells - Sweet, 4",Guinea Fowl
...,...,...,...,...,...,...
995,97928,Tray - 16in Rnd Blk,Pernod,Pastry - Choclate Baked,Ecolab - Lime - A - Way 4/4 L,"Beans - Kidney, Canned"
996,98069,Spinach - Baby,Beef - Ground Medium,Cheese - Parmesan Cubes,Tea - Jasmin Green,Bread - French Baquette
997,98159,Chocolate - Compound Coating,Apricots - Halves,Mustard Prepared,Tea - Jasmin Green,Bread Crumbs - Panko
998,98185,Coffee - Irish Cream,Soup V8 Roasted Red Pepper,Otomegusa Dashi Konbu,"Nut - Pistachio, Shelled",Muffin - Carrot Individual Wrap


In [37]:
cosine=pd.DataFrame(1/(1 + squareform(pdist(matrix.T, 'cosine'))), index=matrix.columns, columns=matrix.columns)
cosine

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.530003,0.563492,0.543546,0.542393,0.541054,0.522016,0.530783,0.533096,0.535293,...,0.530306,0.517987,0.527662,0.511471,0.534181,0.513809,0.528795,0.520208,0.529481,0.527142
200,0.530003,1.000000,0.537166,0.535098,0.521226,0.519193,0.556945,0.516729,0.536350,0.541451,...,0.545224,0.533067,0.555246,0.543554,0.530308,0.521925,0.573177,0.537284,0.526167,0.527280
264,0.563492,0.537166,1.000000,0.531963,0.550168,0.525368,0.538967,0.514932,0.537046,0.524193,...,0.546880,0.525550,0.535062,0.531806,0.553085,0.560225,0.548538,0.538340,0.554929,0.527142
356,0.543546,0.535098,0.531963,1.000000,0.525982,0.523986,0.536792,0.540890,0.516901,0.519482,...,0.536321,0.520565,0.533119,0.534066,0.528874,0.530406,0.520138,0.536202,0.544727,0.535974
412,0.542393,0.521226,0.550168,0.525982,1.000000,0.534073,0.542308,0.547743,0.530459,0.543050,...,0.527903,0.565369,0.528847,0.529278,0.541674,0.536686,0.519628,0.538710,0.546943,0.552636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.513809,0.521925,0.560225,0.530406,0.536686,0.542873,0.540381,0.533190,0.535224,0.526495,...,0.532674,0.539309,0.522883,0.542570,0.569539,1.000000,0.562050,0.543657,0.531363,0.519127
98069,0.528795,0.573177,0.548538,0.520138,0.519628,0.520835,0.534221,0.525789,0.531918,0.541402,...,0.529233,0.535979,0.530224,0.518742,0.554911,0.562050,1.000000,0.548011,0.542143,0.536728
98159,0.520208,0.537284,0.538340,0.536202,0.538710,0.533856,0.535601,0.535704,0.544926,0.535721,...,0.542777,0.556893,0.535487,0.521507,0.541404,0.543657,0.548011,1.000000,0.543258,0.531447


In [38]:
dict_recommendation={}
ls_custID=list(df.CustomerID.unique())
for customer in ls_custID:
    
    similarities = list(cosine[customer].sort_values(ascending=False)[1:].head().index)
    selected_customers=df[df['CustomerID'].isin(similarities)]
    rank_products=selected_customers.groupby('ProductName')['Quantity'].sum().sort_values(ascending=False).reset_index()
    merged=rank_products.merge(matrix, on='ProductName')
    recommended=merged[['ProductName', 'Quantity', customer]].loc[merged[customer]==0].sort_values(by='Quantity', ascending=False)[:5]
  
    dict_recommendation[customer]= list(recommended.ProductName)
df_recom=pd.DataFrame.from_dict(dict_recommendation, orient='index').reset_index()
df_recom.columns=['CustomerID', 'Rec1', 'Rec2', 'Rec3', 'Rec4', 'Rec5']
df_recom

,CustomerID,Rec1,Rec2,Rec3,Rec4,Rec5
0,33,Knife Plastic - White,"Soup - Campbells, Beef Barley",Onions - Cippolini,Tea - Herbal Sweet Dreams,Muffin - Zero Transfat
1,200,Snapple Lemon Tea,Longos - Grilled Salmon With Bbq,General Purpose Trigger,"Thyme - Lemon, Fresh",Tomatoes Tear Drop
2,264,Pickerel - Fillets,"Water - Mineral, Natural",Snapple - Iced Tea Peach,Wine - Ej Gallo Sierra Valley,French Pastry - Mini Chocolate
3,356,Bread - English Muffin,Olive - Spread Tapenade,Bagel - Plain,Pork - Hock And Feet Attached,Wiberg Super Cure
4,412,"Salsify, Organic",Durian Fruit,Wine - Hardys Bankside Shiraz,Bread - Raisin Walnut Oval,Gatorade - Xfactor Berry
...,...,...,...,...,...,...
995,97928,"Spoon - Soup, Plastic",Beef - Montreal Smoked Brisket,Blackberries,Cheese Cloth No 100,"Soup - Campbells, Lentil"
996,98069,Sprouts - Baby Pea Tendrils,Muffin Batt - Blueberry Passion,Juice - Orange,Flavouring - Orange,"Nut - Chestnuts, Whole"
997,98159,"Water, Tap",Bananas,Wine - Redchard Merritt,"Fish - Scallops, Cold Smoked",Cumin - Whole
998,98185,"Peas - Pigeon, Dry",Cheese - Taleggio D.o.p.,Squid U5 - Thailand,"Cheese - Brie, Triple Creme",Pernod


In [44]:
df_recom.loc[df_recom['CustomerID']==200]

,CustomerID,Rec1,Rec2,Rec3,Rec4,Rec5
1,200,Snapple Lemon Tea,Longos - Grilled Salmon With Bbq,General Purpose Trigger,"Thyme - Lemon, Fresh",Tomatoes Tear Drop


In [49]:
recs=pd.DataFrame.from_dict({'euclidean': ['Soup - Campbells Bean Medley', 'Bay Leaf', 'Pork - Kidney', 'Wanton Wrap', 'Muffin - Carrot Individual Wrap'],
                 'correlation': ['Otomegusa Dashi Konbu','Potatoes - Idaho 100 Count','Pail With Metal Handle 16l White','Crackers - Trio','Milk Powder'], 
                             'cityblock': ['Scampi Tail','Pork - Kidney','Muffin - Carrot Individual Wrap','Soup - Campbells Bean Medley','Bay Leaf'],
                            'jaccard': ['WrapBeans - Kidney', 'Muffin - Carrot Individual' , 'Canned Pomello','Chips Potato Salt Vinegar 43g','Wine - Blue Nun Qualitatswein'],
                            'cosine': ['Snapple Lemon Tea', 'Longos - Grilled Salmon With Bbq', 'General Purpose Trigger', 'Thyme - Lemon, Fresh', 'Tomatoes Tear Drop']})

In [50]:
recs
#for only one customer ID 200 are very different depending on the method of calculating the distance

,euclidean,correlation,cityblock,jaccard,cosine
0,Soup - Campbells Bean Medley,Otomegusa Dashi Konbu,Scampi Tail,WrapBeans - Kidney,Snapple Lemon Tea
1,Bay Leaf,Potatoes - Idaho 100 Count,Pork - Kidney,Muffin - Carrot Individual,Longos - Grilled Salmon With Bbq
2,Pork - Kidney,Pail With Metal Handle 16l White,Muffin - Carrot Individual Wrap,Canned Pomello,General Purpose Trigger
3,Wanton Wrap,Crackers - Trio,Soup - Campbells Bean Medley,Chips Potato Salt Vinegar 43g,"Thyme - Lemon, Fresh"
4,Muffin - Carrot Individual Wrap,Milk Powder,Bay Leaf,Wine - Blue Nun Qualitatswein,Tomatoes Tear Drop


euclidian and cityblock have 4 similar proposals, in different order.
